In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py


## Parse ITJ process

In [3]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


## Construct PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

In [7]:
debug = 0
verbose = 1
use_partial_order = 1
reset_to_home = 0

In [8]:
pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug, 
                                            reset_to_home=reset_to_home)

In [9]:
print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)

Init: [('RobotConf', Conf-Home), ('RobotAtConf', Conf-Home), ('CanFreeMove',), ('RobotToolChangerEmpty',), ('Element', 'b0'), ('AtRack', 'b0'), ('IsElement', 'b0'), ('Grounded', 'b0'), ('Element', 'b1'), ('AtRack', 'b1'), ('IsElement', 'b1'), ('Grounded', 'b1'), ('Element', 'b2'), ('AtRack', 'b2'), ('IsElement', 'b2'), ('Element', 'b3'), ('AtRack', 'b3'), ('IsElement', 'b3'), ('Element', 'b5'), ('AtRack', 'b5'), ('IsElement', 'b5'), ('Grounded', 'b5'), ('Element', 'b6'), ('AtRack', 'b6'), ('IsElement', 'b6'), ('Element', 'b7'), ('AtRack', 'b7'), ('IsElement', 'b7'), ('Grounded', 'b7'), ('Element', 'b8'), ('AtRack', 'b8'), ('IsElement', 'b8'), ('Element', 'b4'), ('AtRack', 'b4'), ('IsElement', 'b4'), ('Joint', 'b0', 'b2'), ('Joint', 'b2', 'b0'), ('NoToolAtJoint', 'b0', 'b2'), ('NoToolAtJoint', 'b2', 'b0'), ('Joint', 'b0', 'b3'), ('Joint', 'b3', 'b0'), ('NoToolAtJoint', 'b0', 'b3'), ('NoToolAtJoint', 'b3', 'b0'), ('Joint', 'b0', 'b6'), ('Joint', 'b6', 'b0'), ('NoToolAtJoint', 'b0', 'b6')

In [10]:
from integral_timber_joints.assembly.beam_assembly_method import BeamAssemblyMethod

beam_seq = process.assembly.sequence
for e in beam_seq:
    if process.assembly.get_assembly_method(e) == BeamAssemblyMethod.GROUND_CONTACT:
        print('Grounded: ', e)
for j in process.assembly.joint_ids():
    print(j)

Grounded:  b0
Grounded:  b1
Grounded:  b5
Grounded:  b7
('b0', 'b2')
('b0', 'b3')
('b0', 'b6')
('b1', 'b2')
('b1', 'b3')
('b1', 'b8')
('b2', 'b0')
('b2', 'b1')
('b3', 'b0')
('b3', 'b1')
('b4', 'b5')
('b4', 'b7')
('b5', 'b4')
('b5', 'b6')
('b6', 'b0')
('b6', 'b5')
('b7', 'b4')
('b7', 'b8')
('b8', 'b1')
('b8', 'b7')


## Solve

In [15]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

pddlstream_problem = get_pddlstream_problem(process, use_partial_order=1, debug=True,
                                            reset_to_home=1)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(pddlstream_problem, algorithm='incremental', #incremental
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'c1'), ('AtRack', 'c2'), ('AtRack', 'c3'), ('AtRack', 'c4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'), ('RobotAtConf', Conf-Home))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 115 | Solved: False | Cost: inf | Search Time: 0.016 | Sample Time: 0.000 | Time: 0.016
Iteration: 2 | Complexity: 1 | Calls: 33 | Evaluations: 245 | Solved: False | Cost: inf | Search Time: 0.289 | Sample Time: 0.016 | Time: 0.305
Summary: {complexity: 2, cost: 56.000, evaluations: 245, iterations: 2, length: 56, run_time: 1.277, sample_time: 0.016, search_time: 1.262, solutions: 1, solved: True, timeout: False}
Planning succeeds
----------
 1) pick_gripper_from_rack g1 @conf158 @conf258 @traj60
 2) pick_element_from_rack b0 @conf132 @conf232 @traj34 g1
 3) place_element_on_stru

In [17]:
place_actions = [action for action in plan if action.name == 'place_element_on_structure']
element_sequence = [ac.args[0] for ac in actions]

In [21]:
plan[0]

Action(name='pick_gripper_from_rack', args=('g1', @conf158, @conf258, @traj60))

In [13]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [14]:
element_sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [18]:
place_actions

[Action(name='place_element_on_structure', args=('b0', @conf133, @conf233, @traj35, 'g1')),
 Action(name='place_element_on_structure', args=('b1', @conf135, @conf235, @traj37, 'g1')),
 Action(name='place_element_on_structure', args=('b2', @conf137, @conf237, @traj39, 'g1')),
 Action(name='place_element_on_structure', args=('b3', @conf139, @conf239, @traj41, 'g1')),
 Action(name='place_element_on_structure', args=('b5', @conf141, @conf241, @traj43, 'g1')),
 Action(name='place_element_on_structure', args=('b6', @conf143, @conf243, @traj45, 'g1')),
 Action(name='place_element_on_structure', args=('b7', @conf145, @conf245, @traj47, 'g1')),
 Action(name='place_element_on_structure', args=('b8', @conf147, @conf247, @traj49, 'g1')),
 Action(name='place_element_on_structure', args=('b4', @conf149, @conf249, @traj51, 'g1'))]

# Visualize sequence in a pybullet window

In [ ]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [ ]:
client.disconnect()

In [ ]:
from integral_timber_joints.planning.state import set_state
set_state(client, robot, process, process.initial_state, initialize=False)

# Misc

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)